In [38]:
from bs4 import BeautifulSoup
import urllib.request

from selenium import webdriver
import pandas as pd
import numpy as np

from collections import OrderedDict

import time
from tqdm import tqdm

from nltk.tokenize import word_tokenize



In [39]:
vocab = {} 
preprocessed_sentences = []

# **공시가 발생한 회사명, 종목코드 추출**

1. step one : 첨부서류 코드 모으기
2. step two : 첨부서류에 들어가서 회사명, 종목코드 추출

## **step one - 첨부서류 코드 수집**

In [40]:
docscode_2 = []
cpy_info = []
link_box = []
date = []
checkbox = [19, 67, 24, 27, 14, 65]
""" 11:대표이사변경 19:사채/대출원리금미지급 67:횡령 24:손상차손등 27:어음 위·변조
14:벌금/과징금/행정조치 65:회계처리기준위반등 54:차입"""
start = 2010
end = 2018
section = 3

In [41]:
# 회사명 추출
url = "https://kind.krx.co.kr/disclosure/details.do?method=searchDetailsMain#viewer"
driver = webdriver.Chrome('C:/Users/JH/Desktop/chromedriver')
driver.get(url)
driver.find_element_by_id('rKosdaq').click() # 코스닥
driver.find_element_by_xpath('//*[@id="business"]/option[4]').click() # 산업군
driver.find_element_by_xpath('//*[@id="bfrDsclsType"]').click() # 공시유형 초기화 체크 해제
driver.find_element_by_xpath('//*[@id="dsclsType01"]').click() # 수시공시 클릭

for i in range(len(checkbox)):
    driver.find_element_by_xpath('//*[@id="dsclsLayer01_{num}"]'.format(num = checkbox[i])).click() # 횡령
    #time.sleep(0.1)

time.sleep(0.1)    
driver.find_element_by_xpath('//*[@id="currentPageSize"]/option[4]').click() # 100개씩 보기 선택
time.sleep(0.1)

for i in tqdm(range((end - start)//section + 1)):
    
    sty = driver.find_element_by_id("fromDate")
    #time.sleep(0.1)
    sty.clear()
    #time.sleep(0.1)
    sty.send_keys("{fromD}-01-01".format(fromD= start))
    #time.sleep(0.1)

    edy = driver.find_element_by_id("toDate")
    #time.sleep(0.1)
    edy.clear()
    #time.sleep(0.1)
    edy.send_keys("{toD}-12-31".format(toD = start + 2))

    driver.find_element_by_xpath('//*[@id="searchForm"]/section[1]/div/div[3]/a[1]').click()
    time.sleep(2)

    page = driver.find_element_by_xpath('//*[@id="main-contents"]/section[2]/div[2]/em').text # 총 몇 페이지
    page = page.replace(',','')
    page = int(page)
    page_ = page
    print('{start}-{end_}년도의 해당 공시자료 수는 {page}입니다.'.format(start=start,end_=start+section-1, page=page))
    
    # 남은 페이지가 100이 넘을 때 
    if page <= 1000:
        if page_ >= 100:
            print('1.남은 페이지 수는 {num}입니다.'.format(num = page_))
            
            for i in range(100): # 공시 날짜 수집
                date_value = driver.find_element_by_xpath('//*[@id="main-contents"]/section[1]/table/tbody/tr[{num}]/td[2]'.format(num = i + 1)).text
                date.append(date_value)

            for i in range(100): # 문서 코드
                onclick_value = driver.find_element_by_xpath('//*[@id="main-contents"]/section[1]/table/tbody/tr[{num}]/td[4]/a'.format(num = i + 1)).get_attribute('onclick')
                onclick_value = onclick_value[18:32]
                time.sleep(0.3)
                docscode_2.append(onclick_value)
            page_ -= 100
            
            time.sleep(0.2)
            driver.find_element_by_xpath('//*[@id="main-contents"]/section[2]/div[1]/a[{num}]'.format(num = page//100 + 4)).click()
            time.sleep(2)
    else:
        print('공시문서가 1000개가 넘어 오류가 발생합니다!!')

    # 남은 페이지가 100보다 작을 떄     
    if page_ < 100:
        print('2.남은 페이지 수는 {num}입니다.'.format(num = page_))

        for i in range(page_): # 공시 날짜 수집
            date_value = driver.find_element_by_xpath('//*[@id="main-contents"]/section[1]/table/tbody/tr[{num}]/td[2]'.format(num = i + 1)).text
            date.append(date_value)
        print('잔여 날짜 RESPONSE')

        for i in range(page_): # 문서 코드
            onclick_value = driver.find_element_by_xpath('//*[@id="main-contents"]/section[1]/table/tbody/tr[{num}]/td[4]/a'.format(num = i + 1)).get_attribute('onclick')
            onclick_value = onclick_value[18:32]
            time.sleep(0.3)
            docscode_2.append(onclick_value)
        print('잔여 문서 코드 RESPONSE')

    else:
        print('페이지를 나누는 과정에서 오류가 발생합니다!!')    
    start += 3


  0%|          | 0/3 [00:00<?, ?it/s]

2010-2012년도의 해당 공시자료 수는 131입니다.
1.남은 페이지 수는 131입니다.
2.남은 페이지 수는 31입니다.
잔여 날짜 RESPONSE


 33%|███▎      | 1/3 [00:55<01:50, 55.37s/it]

잔여 문서 코드 RESPONSE
2013-2015년도의 해당 공시자료 수는 133입니다.
1.남은 페이지 수는 133입니다.
2.남은 페이지 수는 33입니다.
잔여 날짜 RESPONSE


 67%|██████▋   | 2/3 [01:51<00:55, 55.84s/it]

잔여 문서 코드 RESPONSE
2016-2018년도의 해당 공시자료 수는 105입니다.
1.남은 페이지 수는 105입니다.
2.남은 페이지 수는 5입니다.
잔여 날짜 RESPONSE


100%|██████████| 3/3 [02:36<00:00, 52.25s/it]

잔여 문서 코드 RESPONSE


## **step two - 해당 공시문서가 발생한 회사명, 종목코드 추출**

In [44]:
for j in range(len(docscode_2)):
        url = 'https://kind.krx.co.kr/common/disclsviewer.do?method=search&acptno={0}&docno=&viewerhost=&viewerport='.format(int(docscode_2[j]))
        driver.get(url)
        time.sleep(0.3)
        cpy_nameAndcode = driver.find_element_by_xpath('//*[@id="frm"]/header/h1').text
        cpy_info.append(cpy_nameAndcode)

cpy_info
driver.close()

In [162]:
""" # 회사명 추출
url = "https://kind.krx.co.kr/disclosure/details.do?method=searchDetailsMain#viewer"
driver = webdriver.Chrome('C:/Users/JH/Desktop/chromedriver')
driver.get(url)
driver.find_element_by_id('rKosdaq').click() # 코스닥
#time.sleep(0.1)
driver.find_element_by_xpath('//*[@id="business"]/option[4]').click() # 산업군
#time.sleep(0.1)
driver.find_element_by_xpath('//*[@id="bfrDsclsType"]').click() # 공시유형 초기화 체크 해제
#time.sleep(0.1)
driver.find_element_by_xpath('//*[@id="dsclsType01"]').click() # 수시공시 클릭
#time.sleep(0.1)

for i in range(len(checkbox)):
    driver.find_element_by_xpath('//*[@id="dsclsLayer01_{num}"]'.format(num = checkbox[i])).click() # 횡령
    #time.sleep(0.1)

time.sleep(0.1)    
driver.find_element_by_xpath('//*[@id="currentPageSize"]/option[4]').click() # 100개씩 보기 선택
time.sleep(0.1)

start = 2015
end = 2020
section = 1

open_code_2 = []
cpy_box = []
for i in tqdm(range((end - start)//section + 1)):
    
    sty = driver.find_element_by_id("fromDate")
    #time.sleep(0.1)
    sty.clear()
    #time.sleep(0.1)
    sty.send_keys("{fromD}-01-01".format(fromD= start))
    #time.sleep(0.1)

    edy = driver.find_element_by_id("toDate")
    #time.sleep(0.1)
    edy.clear()
    #time.sleep(0.1)
    edy.send_keys("{toD}-12-31".format(toD = start))

    driver.find_element_by_xpath('//*[@id="searchForm"]/section[1]/div/div[3]/a[1]').click()
    time.sleep(2)

    page = driver.find_element_by_xpath('//*[@id="main-contents"]/section[2]/div[2]/em').text # 총 몇 페이지
    page = page.replace(',','')
    page = int(page)
    print('{start}년도의 해당 공시자료 수는 {page}입니다.'.format(start=start,page=page))

    for i in range(page):
        onclick_value = driver.find_element_by_xpath('//*[@id="main-contents"]/section[1]/table/tbody/tr[{0}]/td[4]/a'.format(i + 1)).get_attribute('onclick')
        onclick_value = onclick_value[18:32]
        time.sleep(0.3)
        open_code_2.append(onclick_value)

    for j in range(page):
        url = 'https://kind.krx.co.kr/common/disclsviewer.do?method=search&acptno={0}&docno=&viewerhost=&viewerport='.format(int(open_code_2[j]))
        time.sleep(0.3)
        driver.get(url)
        time.sleep(0.3)
        cpy_name = driver.find_element_by_xpath('//*[@id="frm"]/header/h1').text
        cpy_box.append(cpy_name)
        time.sleep(0.3)

    start += 1
    driver.close()
    open_code_2 = [] # 오픈코드 초기화

    url = "https://kind.krx.co.kr/disclosure/details.do?method=searchDetailsMain#viewer"
    driver = webdriver.Chrome('C:/Users/JH/Desktop/chromedriver')
    driver.get(url)
    driver.find_element_by_id('rKosdaq').click() # 코스닥
    #time.sleep(0.1)
    driver.find_element_by_xpath('//*[@id="business"]/option[4]').click() # 산업군
    #time.sleep(0.1)
    driver.find_element_by_xpath('//*[@id="bfrDsclsType"]').click() # 공시유형 초기화 체크 해제
    #time.sleep(0.1)
    driver.find_element_by_xpath('//*[@id="dsclsType01"]').click() # 수시공시 클릭
    #time.sleep(0.1)

    checkbox = [19,14]
    # 67 : 횡령
    for i in range(len(checkbox)):
        driver.find_element_by_xpath('//*[@id="dsclsLayer01_{num}"]'.format(num = checkbox[i])).click() # 횡령
        #time.sleep(0.1)
    
    time.sleep(0.1)    
    driver.find_element_by_xpath('//*[@id="currentPageSize"]/option[4]').click() # 100개씩 보기 선택
    time.sleep(0.1)

print(cpy_box)
print('-'*130)
print(len(cpy_box)) """


' # 회사명 추출\nurl = "https://kind.krx.co.kr/disclosure/details.do?method=searchDetailsMain#viewer"\ndriver = webdriver.Chrome(\'C:/Users/JH/Desktop/chromedriver\')\ndriver.get(url)\ndriver.find_element_by_id(\'rKosdaq\').click() # 코스닥\n#time.sleep(0.1)\ndriver.find_element_by_xpath(\'//*[@id="business"]/option[4]\').click() # 산업군\n#time.sleep(0.1)\ndriver.find_element_by_xpath(\'//*[@id="bfrDsclsType"]\').click() # 공시유형 초기화 체크 해제\n#time.sleep(0.1)\ndriver.find_element_by_xpath(\'//*[@id="dsclsType01"]\').click() # 수시공시 클릭\n#time.sleep(0.1)\n\nfor i in range(len(checkbox)):\n    driver.find_element_by_xpath(\'//*[@id="dsclsLayer01_{num}"]\'.format(num = checkbox[i])).click() # 횡령\n    #time.sleep(0.1)\n\ntime.sleep(0.1)    \ndriver.find_element_by_xpath(\'//*[@id="currentPageSize"]/option[4]\').click() # 100개씩 보기 선택\ntime.sleep(0.1)\n\nstart = 2015\nend = 2020\nsection = 1\n\nopen_code_2 = []\ncpy_box = []\nfor i in tqdm(range((end - start)//section + 1)):\n    \n    sty = driver.find_ele

# **공시문서 텍스트 수집**

In [80]:
driver = webdriver.Chrome('C:/Users/JH/Desktop/chromedriver')
for i in range(int(len(docscode_2)/3)): # 텍스트 링크 추출
    url = 'https://kind.krx.co.kr/common/disclsviewer.do?method=search&acptno={0}&docno=&viewerhost=&viewerport='.format(int(docscode_2[i]))
    driver.get(url)
    time.sleep(0.5)
    link = driver.find_element_by_id('docLocPath').get_attribute('value')
    link_box.append(link)

time.sleep(1)

for i in range(int(len(docscode_2)/3), int(len(docscode_2)/3*2)):
    url = 'https://kind.krx.co.kr/common/disclsviewer.do?method=search&acptno={0}&docno=&viewerhost=&viewerport='.format(int(docscode_2[i]))
    driver.get(url)
    time.sleep(0.5)
    link = driver.find_element_by_id('docLocPath').get_attribute('value')
    link_box.append(link)    

time.sleep(1)

for i in range(int(len(docscode_2)/3*2), len(docscode_2)):
    url = 'https://kind.krx.co.kr/common/disclsviewer.do?method=search&acptno={0}&docno=&viewerhost=&viewerport='.format(int(docscode_2[i]))
    driver.get(url)
    time.sleep(0.5)
    link = driver.find_element_by_id('docLocPath').get_attribute('value')
    link_box.append(link)   

driver.close()
len(link_box)


379

## **공시 문서 URL 리스트에 결측지 유무 확인**

In [93]:
cnt = 0
dftbox = []
for i in range(len(link_box)):
    
    if link_box[i] == '':
        print('index:{num} 주소값에는 결측지가 전달되었습니다.'.format(num = i))
        dftbox.append(i)
        cnt += 1
print('총 결측지는 {num}개 입니다.'.format(num = cnt))

index:186 주소값에는 결측지가 전달되었습니다.
총 결측지는 1개 입니다.


In [123]:
driver = webdriver.Chrome('C:/Users/JH/Desktop/chromedriver')

for i in dftbox:
    url = 'https://kind.krx.co.kr/common/disclsviewer.do?method=search&acptno={num}&docno=&viewerhost=&viewerport='.format(num = int(docscode_2[i]))
    driver.get(url)
    time.sleep(0.5)
    link_box[i] = driver.find_element_by_id('docLocPath').get_attribute('value')

driver.close()

## **결측치 재확인**

In [124]:
cnt_ = 0
for i in range(len(link_box)):
    
    if link_box[i] == '':
        print('index:{num} 주소값에는 결측지가 전달되었습니다.'.format(num = i))
        cnt_ += 1
print('총 결측지는 {num}개 입니다.'.format(num = cnt_))

총 결측지는 0개 입니다.


## **공시 text 추출**

In [125]:
textbox = []

for i in tqdm(range(len(link_box))):
    html = urllib.request.urlopen(link_box[i])

    soup = BeautifulSoup(html, 'html.parser').text
    textbox.append(soup)

print(len(link_box))    


100%|██████████| 379/379 [01:49<00:00,  3.46it/s]

379


['\n\n\n\n\n\n:: 70344_사채 원리금 미지급 발생\n\n\n\n\n\n\n\n\n사채 원리금 미지급 발생\n\n\n\n\n\n\n\n\n1. 미지급금액\n\n\n원금(원)\n\n\n1,392,805,650\n\n\n\n\n이자(원)\n\n\n-\n\n\n\n\n합계(원)\n\n\n1,392,805,650\n\n\n\n\n자기자본(원)\n\n\n11,866,332,009\n\n\n\n\n자기자본대비(%)\n\n\n11.74\n\n\n\n\n대규모 법인 여부\n\n\n미해당\n\n\n\n\n2. 미지급 발생일자\n\n\n2012-11-26\n\n\n\n\n3. 미지급사유 및 대책\n\n\n1) 사유 : 사채권자와의 협의하에 상환기일 연장에 따른 사유 발생\n         \n2) 대책 :사채권자와 협의를 통해 만기도래 사채에 대한 이자액만 지급하고 상환기일(2013년 2월 25일) 연장\n\n\n\n\n4. 기타 투자판단에 참고할 사항\n\n\n-\n\n\n\n\n※관련공시\n\n\n2009.11.23 주요사항보고서(신주인수권부사채권발행결정)\n\n\n\n\n미지급된 사채원리금 내역\n\n\n\n\n사채종류\n\n\n발행회차\n\n\n권면총액(원)\n\n\n발행일\n\n\n미지급 발생일자\n\n\n미지급금액\n\n\n비고\n\n\n\n\n원금(원)\n\n\n이자(원)\n\n\n합계(원)\n\n\n\n\n무기명식 이권부 무보증 분리형 사모 신주인수권부사채\n\n\n4\n\n\n5,000,000,000\n\n\n2009-11-25\n\n\n2012-11-26\n\n\n1,392,805,650\n\n\n-\n\n\n1,392,805,650\n\n\n-\n\n\n\n\n\n\n\n',
 '\n\n\n\n\n\n:: 70345_대출원리금 연체사실 발생\n\n\n\n\n\n\n\n\n대출원리금 연체사실 발생\n\n\n\n\n\n\n\n\n1. 대출원리금 연체금액\n\n\n대출금(원)\n\n\n88,086,131,223\n\n\n\n\n이자(원)\n\n\n0

# **회사명, 코드 및 텍스트 DF, csv로 만들기**

In [126]:
cpyname = []
cpycode = []

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

con1 = OrderedDict([("(", ""), (")", "")])

In [127]:
for i in range(len(cpy_info)):
    infm = cpy_info[i].split(' ')
    cpyname.append(infm[0])

    cpycode.append(replace_all(infm[1] , con1))

In [128]:
text_df = pd.DataFrame({'회사명': cpyname,
                        '거래소코드' : cpycode,
                        '공시발생일' : date,
                        'text' : textbox, 
                        'link' : link_box})
text_df

,회사명,거래소코드,공시발생일,text,link
0,유일엔시스,038720,2012-11-26 17:16,\n\n\n\n\n\n:: 70344_사채 원리금 미지급 발생\n\n\n\n\n\n...,https://kind.krx.co.kr/external/2012/11/26/000...
1,엘컴텍,037950,2012-11-22 15:18,\n\n\n\n\n\n:: 70345_대출원리금 연체사실 발생\n\n\n\n\n\n...,https://kind.krx.co.kr/external/2012/11/22/000...
2,엘컴텍,037950,2012-10-31 18:06,\n\n\n\n\n\n:: 70345_대출원리금 연체사실 발생\n\n\n\n\n\n...,https://kind.krx.co.kr/external/2012/10/31/000...
3,엘컴텍,037950,2012-10-16 17:47,\n\n\n\n\n\n:: 70345_대출원리금 연체사실 발생\n\n\n\n\n\n...,https://kind.krx.co.kr/external/2012/10/16/000...
4,렉스엘이앤지,004790,2012-10-05 17:11,\n\n\n\n\n\n:: 70345_대출원리금 연체사실 발생\n\n\n\n\n\n...,https://kind.krx.co.kr/external/2012/10/05/000...
...,...,...,...,...,...
374,오상헬스케어,036220,2016-02-01 19:08,\n\n\n\n\n\n:: 70008_횡령ㆍ배임 혐의발생\n\n\n\n\n\n\n\...,https://kind.krx.co.kr/external/2016/02/01/001...
375,플렉스컴,065270,2016-01-27 07:12,\n\n\n\n\n\n:: 70345_대출원리금 연체사실 발생\n\n\n\n\n\n...,https://kind.krx.co.kr/external/2016/01/27/000...
376,플렉스컴,065270,2016-01-22 18:48,\n\n\n\n\n\n:: 70345_대출원리금 연체사실 발생\n\n\n\n\n\n...,https://kind.krx.co.kr/external/2016/01/22/000...
377,플렉스컴,065270,2016-01-20 07:15,\n\n\n\n\n\n:: 70345_대출원리금 연체사실 발생\n\n\n\n\n\n...,https://kind.krx.co.kr/external/2016/01/20/000...


## **텍스트 정보 CSV**

In [154]:
text_df.to_csv(r'C:\Users\JH\Desktop\Final\CBA\Code\JH\data\공시67.csv', encoding='utf-8-sig')

# **NLP**

## **토큰화**

In [92]:
wordbox = []

for i in tqdm(range(len(textbox))):
    wordbox.append(word_tokenize(textbox[i]))

wordbox

100%|██████████| 119/119 [00:00<00:00, 526.69it/s]


In [95]:
len(wordbox)

119

In [97]:
for i in tqdm(range(len(wordbox))):
    # 단어 토큰화
    result = []
    for word in wordbox[i]:        
        if word not in vocab:
            vocab[word] = 0 
        vocab[word] += 1
    preprocessed_sentences.append(result) 

100%|██████████| 119/119 [00:00<00:00, 23941.01it/s]


In [107]:
sorted(vocab.items(), key= lambda x :x[1], reverse= True)

[('.', 4778),
 (')', 3299),
 ('(', 2965),
 ('-', 1420),
 (':', 1342),
 ('원', 1207),
 ('1', 785),
 ('2', 775),
 ('상기', 719),
 (',', 679),
 ('3', 622),
 ('및', 508),
 ('횡령', 502),
 ('4', 498),
 ('진행사항', 438),
 ('등', 424),
 ('자기자본', 400),
 ('혐의', 372),
 ('기타', 358),
 ('사항', 353),
 ('%', 351),
 ('참고할', 343),
 ('투자판단에', 341),
 ('자기자본대비', 330),
 ('5', 322),
 ('발생', 309),
 ('예정입니다', 281),
 ('대표이사', 278),
 ('횡령ㆍ배임', 263),
 ('금액', 261),
 ('미해당', 240),
 ('혐의발생', 237),
 ('대출원리금', 237),
 ('발생금액', 234),
 ('확인일자', 233),
 ('연체', 233),
 ('대기업해당여부', 232),
 ('자기자본은', 229),
 ('※관련공시', 228),
 ('6', 211),
 ('대한', 209),
 ('주', 202),
 ('사고발생내용', 200),
 ('따라', 200),
 ('이자', 194),
 ('*', 181),
 ('향후대책', 173),
 ('배임', 171),
 ('전', 169),
 ('합계', 169),
 ('발생일자', 163),
 ('연체사실', 158),
 ('확인일자는', 157),
 ('당사는', 154),
 ('사고발생일자', 145),
 ('중', 143),
 ('관련', 142),
 ('대책', 138),
 ('前', 135),
 ('가', 133),
 ('벌금', 132),
 ('금액입니다', 126),
 ('본', 125),
 ('2019년', 123),
 ('관련하여', 122),
 ('향후', 122),
 ('후', 121),
 ('대출금', 120)